## Install Libraries

In [13]:
!pip install langchain --quiet
!pip install openai --quiet
!pip install pdf2image --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet

## Load the PDF from an online location

In [16]:
from langchain.document_loaders import OnlinePDFLoader

# If using on your S2 workspace remember to add a firewall exception for https://wolfpaulus.com
loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

data = loader.load()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Check to see how many documents and chars are in the book

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 72790 characters in your document


## Now let's split the data into chunks before we turn them into embeddings

In [18]:
# Split the data
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)

# Check how many documents we have now
print (f"You have {len(texts)} pages")

You have 40 pages


## Let's convert the chunks into embeddings and then store it in a SingleStore table

### Let's create the table first

In [19]:
%%sql

USE winter_wikipedia;
DROP TABLE IF EXISTS my_book;
CREATE TABLE IF NOT EXISTS my_book (
    id INT PRIMARY KEY,
    text TEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
    embedding BLOB
);

In [21]:
import openai

# Check to see if there is an environment variable with your API key,
# if not, use what you put below

OPENAI_API_KEY = os.environ.get(
    'OPENAI_API_KEY',
    'YOUR-KEY-HERE'
)

In [22]:
print(texts[0].__dict__)

{'lc_kwargs': {'page_content': 'Everyone you will ever meet knows something you don’t.\n\n[1]\n\nT H E S TO RY\n\n››\n\nof T H E F I E L D\n\nG U I D E\n\nSeveral years ago we created The Field Guide to Data Science because we wanted to help organizations of all types and sizes. There were countless industry and academic publications describing what Data Science is and why we should care, but very little information was available to explain how to make use of data as a resource. We find that situation to be just as true today as we did two years ago, when we created the first edition of the field guide.\n\nAt Booz Allen Hamilton, we built an industry-leading team of Data Scientists. Over the course of hundreds of analytic challenges for countless clients, we’ve unraveled the DNA of Data Science. Many people have put forth their thoughts on single aspects of Data Science. We believe we can offer a broad perspective on the conceptual models, tradecraft, processes and culture of Data Scie

In [23]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [24]:
from langchain.embeddings import OpenAIEmbeddings

# Initialize the OpenAIEmbeddings
embedder = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

## Now let's add the embeddings to the my_book table. Truncate to make sure we don't overwrite

In [25]:
# Clear the my_book table
db_connection.execute("TRUNCATE TABLE my_book")

# Iterate over the texts
for i, document in enumerate(texts):
    # Extract the text content from the Document
    text_content = document.page_content

    # Convert the text to embeddings
    embedding = embedder.embed_documents([text_content])[0]

    # Insert the text and its embedding into the database
    stmt = """
        INSERT INTO my_book (
            id,
            text,
            embedding
        )
        VALUES (
            %s,
            %s,
            JSON_ARRAY_PACK_F32(%s)
        )
    """

    db_connection.execute(stmt, (int(i), str(text_content), str(embedding)))

In [26]:
%%sql

USE winter_wikipedia;
SELECT JSON_ARRAY_UNPACK_F32(embedding)
FROM my_book
WHERE id = 5;

JSON_ARRAY_UNPACK_F32(embedding)


## Do a semantic search using the DOT_PRODUCT in SingleStore instead and JOIN with other data if needed

In [27]:
# Your query text
query_text = "What is the collect stage of data maturity?"

# Convert the query text to embeddings
query_embedding = embedder.embed_documents([query_text])[0]

# Perform a similarity search against the embeddings
stmt = """
    SELECT
        text,
        DOT_PRODUCT_F32(JSON_ARRAY_PACK_F32(%s), embedding) AS similarity
    FROM my_book
    ORDER BY similarity DESC
    LIMIT 1
"""

results = db_connection.execute(stmt, str(query_embedding))

for row in results:
    print(row[1])
    print(row[0])

0.8848673701286316
All of your initial effort will be focused on identifying and aggregating data. Over time, you will have the data you need and a smaller proportion of your effort can focus on Collect. You can now begin to Describe your data. Note, however, that while the proportion of time spent on Collect goes down dramatically, it never goes away entirely. This is indicative of the four activities outlined earlier – you will continue to Aggregate and Prepare data as new analytic questions arise, additional data is needed and new data sources become available.

Organizations continue to advance in maturity as they move through the stages from Describe to Advise. At each stage they can tackle increasingly complex analytic goals with a wider breadth of analytic capabilities. As described for Collect, each stage never goes away entirely. Instead, the proportion of time spent focused on it goes down and new, more mature activities begin. A brief description of each stage of maturity is

In [28]:
import openai
OPENAI_API_KEY = os.environ.get(
    'OPENAI_API_KEY',
    'YOUR-KEY-HERE'
)

# Assign the API key to the openai module
openai.api_key = OPENAI_API_KEY

# Pass the result as a prompt to ChatGPT
prompt = f"The user asked: {query_text}. The most similar text from the book is: {row[0]}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

# Print the assistant's response
print(response['choices'][0]['message']['content'])

The Collect stage of data maturity involves gathering and aggregating internal or external datasets to enhance or refine raw data, and leveraging basic analytic functions such as counts. As organizations advance through the stages of maturity, the proportion of time spent on collecting data goes down, but it never goes away entirely since new data sources and analytic questions will arise over time, requiring continued aggregation and preparation of data.
